# Task A:

## MongoDB Data Model

* When developing an application, question really boils down to what are requirements of an application. In most applications there are requirements to :
> 1. Input data, perform some processing, save the data,retrieve the data and query the data. There may also be a   
    requirement to generate reports on the data.
> 2. Exchange data between different parts of the system or with external systems.
* In order to achieve the requirements for Item 1 a method of persisting data is required. Typically if the volume of data is very small and the type of data is simple and does not require extensive search capabilities then a simple file structure may be used. As data becomes more big and complex, which we have as the old techniques have failed to provide the results on time to predict fire, a XML (or even JSON) structure may be used with the data still stored in files. Searching though becomes more problematic. As the volume of data increases and the complexity of searches increases a database is normally selected which provides industry standard methods for data persistence, querying etc. The databases can designed to handle large volumes of data and store,retrieve and search the data quickly and efficiently.
* In order to achieve the requirements for Item 2 there are various different methods for allowing data interchange between systems including XML, JSON etc.
* Based on the information given in the data sets, climate data is recorded on a daily basis whereas fire data is recorded on the occurence of a fire on a particular day. So, for one climate data, there can be zero or many fire data.
* The list of all the entities in hotspot_historic.csv
> latitude,longitude,datetime,confidence,date,surface_temperature_celcius.
* The list of all the entities in climate_historic.csv
> station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
* There are 2 ways of creating a collection
> 1. Referencing :-  Where _id field of one document is referenced in another document as a reference. For many use cases in MongoDB, the denormalized data model where related data is stored within a single document is optimal
> 2. Embedding :- Embed one document in another.
* Since one to many relationship is a good model design and is using embedding, after the group discussion we decided to go for embedding one document to other.
* Based on the attributes of two data sets provided i.e. hotspot_historic.csv and climate_historic.csv, the two data sets are linked through date attribute from both data sets. For one date there can be zero or many fire  
* We perform a one to many relationship here, on the date attribute. 
* Since we are merging the two datasets, we append the merged entries to a new attribute called Hotspot in the climate dictionary.
> {'air_temperature_celcius': 19,
  'date': '31/12/2016',
  'max_wind_speed': 11.1,
  'precipitation': ' 0.00I',
  'relative_humidity': 56.8,
  'station': 948700,
  'windspeed_knots': 7.9,
  'Hotspot : [{
               'confidence': 87,
               'date': '6/03/2017',
               'datetime': '2017-03-06T05:06:30',
               'location': (-34.3795, 141.6331),
               'surface_temperature_celcius': 62},
              ]'}
* There is no second collection in our database, there is just one collection named 'climate' in our database, since the hotspot_historic dataset is merged to the climate historic. We can retrive the hotspot data just like any other attribute, but here our Hotspot is a list of list(attributes of Hotspot_historic).

In [1]:
# import statements
import pandas as pd
import csv  
import json  
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
#reading in the datasets in pandas`
hotspot_historic = pd.read_csv("hotspot_historic.csv")
climate_historic = pd.read_csv("climate_historic.csv")

In [3]:
#appending to the list hotspot_data
hotspot_data = []
for m,n in hotspot_historic.iterrows():
    hotspot_data.append(list(n.values))
hotspot_data[0][4]

'27/12/2017'

In [4]:
# creating a list called climate_data and appending value to it 
climate_data = []
for m,n in climate_historic.iterrows():
    climate_data.append(list(n.values))
climate_data[0][1]

'31/12/2016'

In [5]:
# creating a list of dictionary inside the dictionary
climate_list = []
#reading the climate_data and saving in to the dictionary called hotspot_list
for i in climate_data:
    dic={}
    dic['station']=i[0]
    dic['date']=i[1]
    dic['air_temperature_celcius']=i[2]
    dic['relative_humidity']=i[3]
    dic['windspeed_knots']=i[4]
    dic['max_wind_speed']=i[5]
    dic['precipitation']=i[6]
    #creating the hotspot_list 
    hotspot_list = []
    #rading the hotspot data and saving into the dictionary called dicti 
    for j in hotspot_data:
        if i[1] == j[4]:
            dicti={}
            dicti['location'] = (j[0],j[1])
            dicti['date'] = j[4]
            dicti['datetime'] = j[2]
            dicti['confidence'] = j[3]
            dicti['surface_temperature_celcius'] = j[5]
            #appending the dictionary into the above created dictionary
            hotspot_list.append(dicti)
    #finally creating hotspot as an attribute in the main dictionary
    dic['Hotspot']=hotspot_list
    climate_list.append(dic)
climate_list

[{'station': 948700,
  'date': '31/12/2016',
  'air_temperature_celcius': 19,
  'relative_humidity': 56.8,
  'windspeed_knots': 7.9,
  'max_wind_speed': 11.1,
  'precipitation': ' 0.00I',
  'Hotspot': []},
 {'station': 948700,
  'date': '2/01/2017',
  'air_temperature_celcius': 15,
  'relative_humidity': 50.7,
  'windspeed_knots': 9.2,
  'max_wind_speed': 13.0,
  'precipitation': ' 0.02G',
  'Hotspot': []},
 {'station': 948700,
  'date': '3/01/2017',
  'air_temperature_celcius': 16,
  'relative_humidity': 53.6,
  'windspeed_knots': 8.1,
  'max_wind_speed': 15.0,
  'precipitation': ' 0.00G',
  'Hotspot': []},
 {'station': 948700,
  'date': '4/01/2017',
  'air_temperature_celcius': 24,
  'relative_humidity': 61.6,
  'windspeed_knots': 7.7,
  'max_wind_speed': 14.0,
  'precipitation': ' 0.00I',
  'Hotspot': []},
 {'station': 948700,
  'date': '5/01/2017',
  'air_temperature_celcius': 24,
  'relative_humidity': 62.3,
  'windspeed_knots': 7.0,
  'max_wind_speed': 13.0,
  'precipitation': ' 

In [6]:
# creating another list for hotspot_list
hotspot_list = []
# appending the hotspot data to the newly created dictionary
for i in hotspot_data:
    dic = {}
    dic['latitude'] = i[0]
    dic['longitude'] = i[1]
    dic['datetime'] = i[2]
    dic['confidence'] = i[3]
    dic['date'] = i[4]
    dic['surface_temperature_celcius'] = i[5]
    
    hotspot_list.append(dic)
len(hotspot_list)

2668

In [7]:
hotspot_list

[{'latitude': -37.966,
  'longitude': 145.05100000000002,
  'datetime': '2017-12-27T04:16:51',
  'confidence': 78,
  'date': '27/12/2017',
  'surface_temperature_celcius': 68},
 {'latitude': -35.541,
  'longitude': 143.311,
  'datetime': '2017-12-27T00:02:15',
  'confidence': 82,
  'date': '27/12/2017',
  'surface_temperature_celcius': 63},
 {'latitude': -35.554,
  'longitude': 143.30700000000002,
  'datetime': '2017-12-27T00:02:15',
  'confidence': 67,
  'date': '27/12/2017',
  'surface_temperature_celcius': 53},
 {'latitude': -35.543,
  'longitude': 143.316,
  'datetime': '2017-12-27T00:02:14',
  'confidence': 86,
  'date': '27/12/2017',
  'surface_temperature_celcius': 67},
 {'latitude': -37.708,
  'longitude': 145.1,
  'datetime': '2017-12-25T04:29:08',
  'confidence': 80,
  'date': '25/12/2017',
  'surface_temperature_celcius': 54},
 {'latitude': -35.646,
  'longitude': 142.282,
  'datetime': '2017-12-24T13:12:01',
  'confidence': 65,
  'date': '24/12/2017',
  'surface_temperature

In [8]:
# importing the MongoClient
from pymongo import MongoClient
#creating the instance of mongoclient
client = MongoClient()
#creating the database called db
db = client.fit5148_assignment_db
#dumping the json in the database
climate_hotspot = json.dumps([row for row in climate_list])
climate_hotspot_json = json.loads(climate_hotspot)

#creating the collection called climate in the database
db.climate.drop()
db.climate.insert_many(climate_hotspot_json)

KeyboardInterrupt: 

In [ ]:
climate_hotspot_json

##### Query A.

In [ ]:
db.climate.find_one({'date': '10/12/2017'})

##### Query B.

In [9]:
results = db.hotspot.find({'surface_temperature_celcius': {'$gte': 65, '$lt': 100}}, ['location', 'surface_temperature_celcius', 'confidence'])
for eachResult in results:
    print(eachResult)


KeyboardInterrupt: 

##### Query C.

In [12]:
results = db.climate.find({'date':{'$in':['15/12/2017','16/12/2017']}},{'Hotspot.surface_temperature_celcius':1,"air_temperature_celcius":1,"relative_humidity":1,"max_wind_speed":1,"date":1,"_id":1})

In [13]:
for eachResult in results:
    pprint(eachResult)

{'Hotspot': [{'surface_temperature_celcius': 42},
             {'surface_temperature_celcius': 36},
             {'surface_temperature_celcius': 38},
             {'surface_temperature_celcius': 40}],
 '_id': ObjectId('5ce685a293436914c6f7fdfe'),
 'air_temperature_celcius': 18,
 'date': '15/12/2017',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'Hotspot': [{'surface_temperature_celcius': 43},
             {'surface_temperature_celcius': 33},
             {'surface_temperature_celcius': 54},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 75},
             {'surface_temperature_celcius': 55},
             {'surface_temperature_celcius': 66},
             {'surface_temperature_celcius': 56},
             {'surface_temperature_celcius': 60},
             {'surface_temperature_celcius': 73},
             {'surface_temperature_celcius': 48},
             {'surface_temperature_celcius': 

##### Query D.

In [18]:
collection_1= db.climate.aggregate([{"$unwind": "$Hotspot"},
                               {"$match":{"Hotspot.confidence":{"$gt":80,"$lt":100}}},
                               {"$project":{"_id":0,"Hotspot.confidence":1,"Hotspot.surface_temperature_celcius":1,"air_temperature_celcius":1,"Hotspot.datetime":1}}]) 
for i in collection_h:
    print(i)

{'Hotspot': {'confidence': 87, 'datetime': '2017-03-06T05:06:30', 'surface_temperature_celcius': 62}, 'air_temperature_celcius': 20}
{'Hotspot': {'confidence': 85, 'datetime': '2017-03-06T05:06:20', 'surface_temperature_celcius': 59}, 'air_temperature_celcius': 20}
{'Hotspot': {'confidence': 88, 'datetime': '2017-03-07T04:16:10', 'surface_temperature_celcius': 64}, 'air_temperature_celcius': 19}
{'Hotspot': {'confidence': 86, 'datetime': '2017-03-09T13:23:40', 'surface_temperature_celcius': 41}, 'air_temperature_celcius': 23}
{'Hotspot': {'confidence': 94, 'datetime': '2017-03-10T04:45:30', 'surface_temperature_celcius': 73}, 'air_temperature_celcius': 19}
{'Hotspot': {'confidence': 97, 'datetime': '2017-03-10T04:45:30', 'surface_temperature_celcius': 80}, 'air_temperature_celcius': 19}
{'Hotspot': {'confidence': 81, 'datetime': '2017-03-10T04:43:50', 'surface_temperature_celcius': 55}, 'air_temperature_celcius': 19}
{'Hotspot': {'confidence': 94, 'datetime': '2017-03-12T04:33:50', 'su

##### Query E.

In [19]:
collection_2=db.climate.aggregate([{"$unwind":"$Hotspot"},{"$sort":{"Hotspot.surface_temperature_celcius":-1}},{"$limit":10}])
for i in collection_2:
    pprint(i)

{'Hotspot': {'confidence': 100,
             'date': '18/04/2017',
             'datetime': '2017-04-18T04:52:00',
             'location': [-38.1665, 143.062],
             'surface_temperature_celcius': 124},
 '_id': ObjectId('5ce685a293436914c6f7fd0d'),
 'air_temperature_celcius': 15,
 'date': '18/04/2017',
 'max_wind_speed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'Hotspot': {'confidence': 100,
             'date': '4/04/2017',
             'datetime': '2017-04-04T04:32:50',
             'location': [-36.343, 142.1986],
             'surface_temperature_celcius': 123},
 '_id': ObjectId('5ce685a293436914c6f7fcff'),
 'air_temperature_celcius': 16,
 'date': '4/04/2017',
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'Hotspot': {'confidence': 100,
             'date': '1/05/2017',
             'datetime': '2017-05-01T04:14:20',
             

##### Query E.

In [24]:
collection_3=db.climate.aggregate([{"$unwind":"$Hotspot"},
                                  {"$group":{"_id":{"datetime":"$datetime","date":"$date","_id":"$_id"},
                                            "count":{"$sum":1}}},
                                  {"$project":{"date":1,"count":1,"_id":0}}])

##### Query F.

In [30]:
collection_4=db.climate.aggregate([{ "$match": {"Hotspot": {"$exists": True, "$ne":[]}}},
                                   {"$project":{"_id":0,"date": 1,
                                                "TotalNo_of_fires":{"$size":"$Hotspot"}}}])

##### Query G.

In [42]:
collection_5=db.climate.aggregate([{"$match": {"Hotspot":{"$exists": True,"$ne":[]}}},
                                  {"$project":{"_id":0,"date":1, "avg":{"$avg":"$Hotspot.surface_temperature_celcius"}}}])